# COVID19 - Countries

In [1]:
from environs import Env
env = Env()
env.str("CUDA_DEVICE_ORDER",'PCI_BUS_ID')
env.int("CUDA_VISIBLE_DEVICES",1)
env.int("NUMBA_ENABLE_CUDASIM",1)
env.bool("OMPI_MCA_opal_cuda_support",True)

import os
import ray
# MB=1024*1024
# GB=MB*1024
# ray.init(num_gpus=5,num_cpus=1)
# @ray.remote(num_gpus=1)
# def use_gpu():
#     print("ray.get_gpu_ids(): {}".format(ray.get_gpu_ids()[0]))
#     print("CUDA_VISIBLE_DEVICES: {}".format(os.environ["CUDA_VISIBLE_DEVICES"]))

# use_gpu.remote()

In [2]:
import urllib.request
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

In [3]:
# Download data
import get_data as gd
LoadData=True

if LoadData:
    gd.get_data()

# SEAIR-D Model Equations

$$\begin{array}{l}\frac{d s}{d t}=-[\beta i(t) + \beta_2 a(t)-\mu] \cdot s(t)\\ 
\frac{d e}{d t}=[\beta i(t) + \beta_2 a(t)] \cdot s(t) -(\sigma+\mu) \cdot e(t)\\ 
\frac{d a}{d t}=\sigma e(t) \cdot (1-p)-(\gamma+\mu) \cdot a(t) \\
\frac{d i}{d t}=\sigma e(t) \cdot p - (\gamma + \sigma_2 + \sigma_3 + \mu) \cdot i(t)\\ 
\frac{d r}{d t}=(b + \sigma_2) \cdot i(t) + \gamma \cdot a(t) - \mu \cdot r(t)\\
\frac{d k}{d t}=(a + \sigma_3 - \mu) \cdot d(t)
\end{array}$$

The sum of all people accounted is equal to population considered:

$$s + e + a + i + r + k = N$$

The derivate is:

$$\frac{d s}{d t}+\frac{d e}{d t}+\frac{d a}{d t}+\frac{d i}{d t}+\frac{d r}{d t}+\frac{d k}{d t} = 0$$

The last equation does not need to be solve because:

$$\frac{d k}{d t}=-(\frac{d s}{d t}+\frac{d e}{d t}+\frac{d a}{d t}+\frac{d i}{d t}+\frac{d r}{d t})$$

The sum of all rates are equal to zero! The importance of this equation is that it conservates the rates.


## Parameters

N: the population considered

$\beta$: Effective contact rate [1/min]
    
$\gamma$: Recovery(+Mortality) rate $\gamma=(a+b)$ [1/min]

$a$: mortality of healed  [1/min]

$b$: recovery rate  [1/min]

$\sigma$: is the rate at which individuals move from the exposed to the infectious classes. Its reciprocal ($1/\sigma$) is the average latent (exposed) period.

$\sigma_2$: is the rate at which individuals move from the infectious to the healed classes. Its reciprocal ($1/\sigma_2$) is the average latent (exposed) period

$\sigma_3$: is the rate at which individuals move from the infectious to the dead classes. Its reciprocal ($1/\sigma_3$) is the average latent (exposed) period
    
$p$: is the fraction of the exposed which become symptomatic infectious sub-population.

$(1-p)$: is the fraction of the exposed which becomes asymptomatic infectious sub-population.

## Options: Param.CSV file, changeCSV, Adjust Param, Save CSV, Select Specific DRS

In [4]:
from IPython.display import display, HTML, Markdown

paramOpt=1
changeCSV=True
adjustParam=True
paramSave=False
selectCountries=False

## Different Options to Load Param.CSV file

In [5]:
if paramOpt==0:
    paramFile="data/param_optimized_Yabox_HistMin-Copy1.csv"
    version = "001"
    model = "YaboxIC"    

if paramOpt==1:
    paramFile="data/param_optimized_Yabox_HistMin.csv"
    version = "003"
    model = "YaboxIC"    

In [6]:
dfparam = pd.read_csv(paramFile)
dfparam = dfparam.dropna()
dfparam.e0=0
dfparam.r0=0

display(Markdown("## Original Initial Parameters"))
display(dfparam)

## Original Initial Parameters

,country,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,WCASES,WREC
0,Brazil,03/05/20,200,14760588,0,0,33,0,121,149,0.2823,0.1813
1,China,01/27/20,200,391756,0,0,2,0,178,99,0.2530,0.2059
2,Italy,02/29/20,200,643531,0,0,191,0,113,131,0.0641,0.1625
3,US,02/22/20,200,85527338,0,0,209,0,185,24,0.2523,0.1744
4,India,03/14/20,200,390053627,0,0,231,0,60,109,0.0677,0.0301


## Modify param.csv file to fit better the data

In [7]:
if changeCSV and (paramOpt==1):
    dfparam['s0'] = pd.to_numeric(dfparam['s0'],errors='coerce')
    dfparam['i0'] = pd.to_numeric(dfparam['i0'],errors='coerce')
    dfparam['d0'] = pd.to_numeric(dfparam['d0'],errors='coerce')
    dfparam['s0'] = dfparam['s0'].astype(float)
    dfparam['i0'] = dfparam['i0'].astype(float)
    dfparam['d0'] = dfparam['d0'].astype(float)
    dfparam.loc[dfparam.country=='Italy','s0'] = dfparam.loc[dfparam.country=='Italy','s0']*1.5
    dfparam.loc[dfparam.country=='China','s0'] = dfparam.loc[dfparam.country=='China','s0']*1.5
    dfparam.loc[dfparam.country=='India','s0'] = dfparam.loc[dfparam.country=='India','s0']*4/100
    dfparam.loc[dfparam.country=='US','s0'] = dfparam.loc[dfparam.country=='US','s0']*2/10
    dfparam['s0'] = pd.to_numeric(dfparam['s0'],errors='coerce')
    dfparam['i0'] = pd.to_numeric(dfparam['i0'],errors='coerce')
    dfparam['d0'] = pd.to_numeric(dfparam['d0'],errors='coerce')
    

# if adjustParam:
#     sCorrect=[1.38,1.0,1.05,1.5,1.5,1.4,1.1,1.5,1.15,1.55,1.35,1.2,1.35,1.15,1.55,1.45,1.25]
#     dfparam.s0=dfparam.s0.multiply(sCorrect[:len(DRS)], axis=0)

if selectCountries and paramOpt==0:
    countries=['Brazil', 
         'US']

if not selectCountries:
    countries=dfparam.country    

dfparam['s0'] = dfparam['s0'].astype(int)
dfparam['i0'] = dfparam['i0'].astype(int)
dfparam['d0'] = dfparam['d0'].astype(int)

if paramSave:
    dfparam.to_csv("new_"+paramFile)    

display(Markdown("## Modified Initial Parameters"))
display(dfparam)

## Modified Initial Parameters

,country,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,WCASES,WREC
0,Brazil,03/05/20,200,14760588,0,0,33,0,121,149,0.2823,0.1813
1,China,01/27/20,200,587634,0,0,2,0,178,99,0.2530,0.2059
2,Italy,02/29/20,200,965296,0,0,191,0,113,131,0.0641,0.1625
3,US,02/22/20,200,17105467,0,0,209,0,185,24,0.2523,0.1744
4,India,03/14/20,200,15602145,0,0,231,0,60,109,0.0677,0.0301


# Solver Loading and Version

In [8]:
%reload_ext autoreload
%autoreload 2
import LearnerYabox_v2 as Learner #Yabox

2020-09-28 13:25:47,808	INFO resource_spec.py:212 -- Starting Ray with 229.98 GiB memory available for workers and up to 73.18 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-09-28 13:25:48,124	INFO services.py:1170 -- View the Ray dashboard at localhost:8266


In [9]:
allCountries=False
cleanRecovered=False
version="1"

if allCountries:
    display(countries)
else:
    countrySelected="Brazil"
    display(countrySelected)

'Brazil'

In [10]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return true;}

<IPython.core.display.Javascript object>

# Main Loop

In [ ]:
results=[]
if allCountries:
    for country in countries:
        query = dfparam.query('country == "{}"'.format(country)).reset_index()
        parameters = np.array(query.iloc[:, 2:])[0]
        current = datetime.strptime(parameters[0], '%m/%d/%y')
        parameters[0] = datetime.strftime(current, '%-m/%-d/%y')
        f=Learner.Learner.remote(country, *parameters, cleanRecovered, version)
        result = f.train.remote() 
        results.append(result)
else:
    country = countrySelected 
    query = dfparam.query('country == "{}"'.format(country)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    current = datetime.strptime(parameters[0], '%m/%d/%y')
    parameters[0] = datetime.strftime(current, '%-m/%-d/%y')
    f=Learner.Learner.remote(country, *parameters, cleanRecovered, version)
    result = f.train.remote() 
    results.append(result)

# #execute all the queue with max_runner_cap at a time    
results = ray.get(results)

  0%|          | 0/21125000.0 [00:00<?, ?it/s]
  0%|          | 0/21125000.0 [00:02<?, ?it/s]
  0%|          | 1/21125000.0 [00:03<7530:14:53,  1.28s/it]
  0%|          | 3/21125000.0 [00:05<6454:46:50,  1.10s/it]
  0%|          | 6/21125000.0 [00:06<5314:41:23,  1.10it/s]
  0%|          | 10/21125000.0 [00:07<4316:59:01,  1.36it/s]
  0%|          | 15/21125000.0 [00:09<3504:10:57,  1.67it/s]
  0%|          | 21/21125000.0 [00:10<2854:34:55,  2.06it/s]
  0%|          | 28/21125000.0 [00:12<2324:52:13,  2.52it/s]
  0%|          | 36/21125000.0 [00:13<1911:33:09,  3.07it/s]
  0%|          | 45/21125000.0 [00:14<1596:22:48,  3.68it/s]
  0%|          | 55/21125000.0 [00:15<1342:33:06,  4.37it/s]
  0%|          | 66/21125000.0 [00:17<1141:35:03,  5.14it/s]
  0%|          | 78/21125000.0 [00:18<989:26:13,  5.93it/s] 
  0%|          | 91/21125000.0 [00:19<863:18:41,  6.80it/s]
  0%|          | 105/21125000.0 [00:21<769:51:34,  7.62it/s]
  0%|          | 120/21125000.0 [00:22<691:59:25,  8.48i

  0%|          | 8911/21125000.0 [02:14<42:29:42, 138.03it/s]
  0%|          | 9045/21125000.0 [02:15<40:35:11, 144.52it/s]
  0%|          | 9180/21125000.0 [02:15<39:08:34, 149.85it/s]
  0%|          | 9316/21125000.0 [02:16<38:04:05, 154.08it/s]
  0%|          | 9453/21125000.0 [02:17<36:47:36, 159.41it/s]
  0%|          | 9591/21125000.0 [02:18<35:57:50, 163.09it/s]
  0%|          | 9730/21125000.0 [02:19<35:27:55, 165.38it/s]
  0%|          | 9870/21125000.0 [02:20<35:01:32, 167.46it/s]
  0%|          | 10011/21125000.0 [02:20<35:10:09, 166.77it/s]
  0%|          | 10153/21125000.0 [02:21<35:01:27, 167.46it/s]
  0%|          | 10296/21125000.0 [02:22<34:10:03, 171.66it/s]
  0%|          | 10440/21125000.0 [02:23<34:17:36, 171.03it/s]
  0%|          | 10585/21125000.0 [02:24<34:20:07, 170.82it/s]
  0%|          | 10731/21125000.0 [02:25<34:27:26, 170.21it/s]
  0%|          | 10878/21125000.0 [02:25<34:00:18, 172.48it/s]
  0%|          | 11026/21125000.0 [02:26<33:24:59, 175.51it/s]


  0%|          | 34980/21125000.0 [04:04<18:47:35, 311.73it/s]
  0%|          | 35245/21125000.0 [04:05<18:44:45, 312.51it/s]
  0%|          | 35511/21125000.0 [04:06<18:33:07, 315.77it/s]
  0%|          | 35778/21125000.0 [04:06<18:26:06, 317.77it/s]
  0%|          | 36046/21125000.0 [04:07<18:38:44, 314.18it/s]
  0%|          | 36315/21125000.0 [04:08<18:55:53, 309.43it/s]
  0%|          | 36585/21125000.0 [04:09<19:09:29, 305.76it/s]
  0%|          | 36856/21125000.0 [04:10<18:58:40, 308.66it/s]
  0%|          | 37128/21125000.0 [04:11<18:49:53, 311.06it/s]
  0%|          | 37401/21125000.0 [04:12<18:37:40, 314.46it/s]
  0%|          | 37675/21125000.0 [04:13<18:24:11, 318.29it/s]
  0%|          | 37950/21125000.0 [04:13<18:12:37, 321.66it/s]
  0%|          | 38226/21125000.0 [04:14<18:08:32, 322.86it/s]
  0%|          | 38503/21125000.0 [04:15<17:57:42, 326.10it/s]
  0%|          | 38781/21125000.0 [04:16<17:58:42, 325.79it/s]
  0%|          | 39060/21125000.0 [04:17<17:57:58, 326.

In [ ]:
ray.shutdown()

# Plots

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
import pandas as pd
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2
import covid_plots_v2 as covid_plots

In [ ]:
def loadDataFrame(filename):
    df= pd.read_pickle(filename)
    df.columns = [c.lower().replace(' ', '_') for c in df.columns]
    df.columns = [c.lower().replace('(', '') for c in df.columns]
    df.columns = [c.lower().replace(')', '') for c in df.columns]
    return df

In [ ]:
#main country for analysis
countryMain = "Brazil"

#select districts for plotting
countries4Plot=['China',
               'US',
               'India',
               'Italy',
               countryMain]

#Choose here your options
#opt=0 all plots
#opt=1 corona log plot
#opt=2 logistic model prediction
#opt=3 bar plot with growth rate
#opt=4 log plot + bar plot
#opt=5 SEAIR-D Model
opt = 5

#number of cases to start plotting model in log graph - real data = 100
startCase=100

#all DRS for plotting
countries=dfparam.country

In [ ]:
#do not allow the scrolling of the plots

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return false;}

In [ ]:
#plots one district or all districts  
if opt==5:
    allCountries=True
else:
    allCountries=False
    
if allCountries:
    for country in countries:
        query = dfparam.query('country == "{}"'.format(country)).reset_index()
        startCase = query['START'][0]
        startdate = query['start-date'][0]
        predict_range = query['prediction-range'][0]
        #calcula data máxima dos gráficos
        #100 dias é usado como máximo dos cálculos da derivada das mortes
        lastDate='9/22/20'
        maxDate= datetime.strptime(lastDate, "%m/%d/%y") + timedelta(days = 100) #"2020-08-31"
        maxDateStr = maxDate.strftime("%-m/%-d/%y")
        covid_plots.covid_plots(country, countries4Plot, startdate, predict_range, \
                        startCase, opt, version, show=True, maxDate=maxDateStr, model=model)
else: 
    query = dfparam.query('country == "{}"'.format(country)).reset_index()
    startdate = query['start-date'][0]
    predict_range = query['prediction-range'][0]
    startCase = query['START'][0]
    #calcula data máxima dos gráficos
    #100 dias é usado como máximo dos cálculos da derivada das mortes
    lastDate='9/22/20'
    maxDate= datetime.strptime(lastDate, "%m/%d/%y") + timedelta(days = 100) #"2020-08-31"
    maxDateStr = maxDate.strftime("%-m/%-d/%y")   
    covid_plots.covid_plots(country, countries4Plot,  startdate, predict_range, \
                       startCase, opt, version, show=True, maxDate=maxDateStr, model=model)